In [1]:
#importing the required libraries
import requests
import re
import csv
import lxml
from bs4 import BeautifulSoup
import pandas as pd
import xml.etree.ElementTree as et
import numpy as np
import yfinance as yf
import json

In [2]:
#global styling display
pd.set_option('display.max_rows',None)
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_colwidth',1000)

In [3]:
#Setting the variables
sec_url = 'https://www.sec.gov'
cusip_url='https://www.sec.gov/data/foiadocsfailsdatahtm'
requested_cik ='0001067983'
doctype='13F-HR'
Num_Hist_Qtrs=9  ##How many filings to scrape

In [4]:
#Creating the functions

def get_request(url):
    return requests.get(url)

def create_url(cik,doctype):
    return 'https://www.sec.gov/cgi-bin/browse-edgar?CIK={}&owner=exclude&action=getcompany&type={}'.format(cik,doctype)

def scrape_data(requested_cik,Qtr):
    #xml_file_position=3 for 13F-HR , =4 for 10-K
    
    #getting the urls of the document(eg.13F-HR) locations
    #which further have html,xml file urls
    doctype = '13F-HR'
    xml_file_position=3
    
    response = get_request(create_url(requested_cik,doctype))
    soup = BeautifulSoup(response.text, "html.parser")
    tags = soup.findAll('a', id="documentsbutton")
    
    #going to a single document location and extracting the xml file location
    #by parsing thru the soup html object
    response_two = get_request(sec_url + tags[Qtr]['href'])
    soup_two = BeautifulSoup(response_two.text, "html.parser")
    tags_two = soup_two.findAll('a', attrs={'href': re.compile('xml')})
    xml_url = tags_two[xml_file_position].get('href')
    response_xml = get_request(sec_url + xml_url)
    soup_xml = BeautifulSoup(response_xml.content, "lxml")
    return soup_two,soup_xml

def table13F(requested_cik,Num_Hist_Qtrs):
    
    appended_table = []
    for Qtr in range(Num_Hist_Qtrs):
    
        soup_two,soup_xml = scrape_data(requested_cik,Qtr)

        #Extracting the filing and reporting date from the FormContent in the soup html object
        table_tag = soup_two.find('div', class_=['formContent'])
        table_tag2 = table_tag.find_all('div', class_=['infoHead','info'])

        FilingDate = table_tag2[1].text
        PeriodofReport = table_tag2[7].text
        
        #Extracting the CIK
        table_tag3 = soup_two.find('div', class_=['companyInfo'])
        table_tag4 = table_tag3.find('a', attrs={'href': re.compile('CIK')}).string
        CIK = table_tag4.split()[0].lstrip('0')

        #Extracting data from the soup xml object
        rows = soup_xml.find_all('infotable')
        positions = []
        for row in rows:
            dic = {}
            position = row.find_all()
            dic["NAME_OF_ISSUER"] = position[0].text
            dic["CUSIP"] = position[2].text
            dic["SHARES"] = float(position[5].text.replace(',', ''))/1000
            dic["PeriodofReport"]= table_tag2[7].text
            positions.append(dic)
        data = pd.DataFrame(positions)
        appended_table.append(data)
    appended_table = pd.concat(appended_table)
#     appended_table =pd.pivot_table(appended_table,index=["NAME_OF_ISSUER","CUSIP"],values=["SHARES"],
#                columns=["PeriodofReport"],aggfunc=[np.sum],fill_value=0)
    return appended_table

In [5]:
#Downloading the CUSIP and Ticker mapping
response = get_request(cusip_url)
soup = BeautifulSoup(response.text, "html.parser")
url_href=soup.findAll('a', attrs={'href': re.compile('cnsfails')})[0]
file_loc = url_href['href']
file_download_url =(sec_url + file_loc)
df = pd.read_csv(file_download_url,sep='|')
df2=df[['CUSIP','SYMBOL']].drop_duplicates()
appended_table=table13F(requested_cik,Num_Hist_Qtrs)

In [27]:
#Merging tables for CUSIP and SYMBOL
Symbol_13F_table = appended_table.merge(df2,how='left',on="CUSIP")

In [33]:
#Getting market data for the SYMBOLS
positions=[]
table_tickers = Symbol_13F_table["SYMBOL"].unique()
for n in range(len(table_tickers)):
    dic = {}
    try:
        yfinanceData = yf.Ticker(table_tickers[n])
        info = yfinanceData.info
        dic["SYMBOL"] = table_tickers[n]
        dic["MarketCap"] = info['marketCap']
        dic["SharesOutstanding"] = info['sharesOutstanding']
    except :
        dic["SharesOutstanding"] = 0
        print(table_tickers[n])
    positions.append(dic)
data = pd.DataFrame(positions)

SPY
VOO
nan


In [58]:
#output
## Merging original 13F data with market info
output = Symbol_13F_table.merge(data,on='SYMBOL')

output_group = output.groupby(by=["NAME_OF_ISSUER","SYMBOL","CUSIP","PeriodofReport"])

final_output = output_group.agg({"SharesOutstanding":'max',"MarketCap":'max',"SHARES":'sum'})
final_output = final_output.stack().unstack("PeriodofReport")

In [74]:
final_output=final_output.fillna("Null")

In [75]:
format_dict = {'SharesOutstanding':'{0:,.0f}', 'MarketCap': '${0:,.0f}', 'SHARES': '{0:,.0f}'}
final_output.style.format(format_dict)